# Data loader

In [1]:
from DataConsumer.comtrade_consumer import ComtradeConsumer
from DataConsumer.wits_consumer import WitsConsumer

from xml.dom.minidom import parse, parseString
import pandas as pd

import time

# General parameters

In [2]:
years = list(range(2000, 2022))

## HHI

#### It's necessary to query all the country availables and then iterate each combination (year, country) for getting all HHI values

In [18]:
wc = WitsConsumer()

In [20]:
list_iso3_codes = wc.get_list_iso3codes();

In [21]:
list_iso3_codes[3:10]

'<?xml v'

In [22]:
s = parseString(list_iso3_codes[3:])

In [23]:
list_xml_countries = s.getElementsByTagName("wits:country")

In [24]:
country_names, iso3_codes = list(), list()
for node in list_xml_countries:
    country_names.append(node.getElementsByTagName("wits:name")[0].firstChild.wholeText)
    iso3_codes.append(node.getElementsByTagName("wits:iso3Code")[0].firstChild.wholeText)

In [25]:
assert len(country_names) == len(iso3_codes)

In [26]:
# Let's iterate over the years and countries to get the hhi index
for year in years:
    year_list4df, country_list4df, iso3_list4df, hhi_list4df = list(), list(), list(), list()

    for country, iso3code in zip(country_names, iso3_codes):
        year_list4df.append(year)
        # country_list4df.append(country)
        # iso3_list4df.append(iso3code)

        try:
            hhi = wc.get_hhi_index(iso3code=iso3code, year=year)
        except Exception:
            hhi = float('nan')
            
        hhi_list4df.append(hhi)
        print(f"{year} \t {iso3code} \t {hhi}")
        # break
        time.sleep(5)
    # break
    stage_df = pd.DataFrame({
        "Year": year_list4df,
        "Country": country_names,
        "iso3code": iso3_codes,
        "hhi": hhi_list4df
    })
    stage_df.to_csv(f"../data/hhi_dfs/stage_{year}.csv", index=False)


[get_hhi_index][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
2016 	 A01 	 None
[get_hhi_index][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
2016 	 A02 	 None
[get_hhi_index][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
2016 	 A03 	 None
[get_hhi_index][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
2016 	 A04 	 None
[get_hhi_index][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
2016 	 A05 	 None
[get_hhi_index][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
2016 	 A07 	 None
[get_hhi_index][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
2016 	 A08 	 None
[get_hhi_index][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
2016 	 A09 	 None
[get_hhi_index][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
2016 	 A10 	 None
[get_hhi_index][ERROR] Something happened: Expecting value: line 1 column

In [37]:
# Merge all the stage files into a single one
final_hhi_df = pd.DataFrame({
                "Year": [],
                "Country": [],
                "iso3code": [],
                "hhi": []
            })

for year in years:
    tmp_df = pd.read_csv(f"../data/hhi_dfs/stage_{year}.csv")
    final_hhi_df = pd.concat([final_hhi_df, tmp_df])

final_hhi_df[["Year", "Country", "iso3code", "hhi"]].to_csv("../data/hhi_dfs/hhi_index_all_years.csv", index=False)

# Imports

In [3]:
ctc = ComtradeConsumer()

In [4]:
# Lista de elementos de interés
classification_codes = {
    "Antimony": 261710,
    "Asbestos": 2524,
    "Barytes": 2511,
    "Bismuth": 8106,
    "Cadmium": 8107,
    "Chromium": 2610,
    "Coal": 2701,
    "Cobalt": 810520,
    "Copper": 2603,
    "Gold": 7108,
    "Graphite": 2504,
    "Iron": 2601,
    "Lead": 2607,
    "Lithium": 283691,
    "Magnesite": 251910,
    "Magnesium": 251910,
    "Manganese": 2602,
    "Mercury": 280540,
    "Molybdenum": 2613,
    "Natural gas": 271111,
    "Nickel": 2604,
    "Petroleum": 2709,
    "Rare earth": 2846,
    "Silver": 261610,
    "Tin": 2609,
    "Titanium": 2614,
    "Tungsten": 2611,
    "Uranium": 261210,
    "Zinc": 2608,
    "Zirconium": 261510,
    "Lithium-ion batteries": 850760 # igual que Litio?
}


In [5]:
# Obtenemos lista de países disponibles para reporter (igual que partner (?))
list_reporter_countries = ctc.get_reporter_countries()
# Remueve "all" porque marca error --> indice 0
list_reporter_countries = list_reporter_countries[1:]


In [6]:
for year in years:
    years_list, reporter_list, reporterCode_list, partner_list, partnerCode_list, cc_list, tradeValue_list, tradeQty_list, TVperUnit_list = [], [], [], [], [], [], [], [], []
    for country in list_reporter_countries:
        for cc in classification_codes.keys():
            datasets = ctc.get_trade_value_imports(
                                    type='C',
                                    freq='A',
                                    px='HS',
                                    ps=year,
                                    r=country['id'],
                                    p='ALL',
                                    rg=1,
                                    cc=classification_codes[cc],
                                )
            
            # Iterate all the trade values available
            # total_tv, total_tqty = float('nan'), float('nan')
            for dataset in datasets:
                tv = float("nan")
                try:
                    tv = dataset['TradeValue'] 
                    # total_tv += tv
                except Exception:
                    pass

                tqty = float('nan')
                try:
                    tqty = dataset['TradeQuantity']
                    # total_tqty += tqty
                except Exception:
                    pass
                
                tv_per_unit = float('nan')
                try:
                    tv_per_unit = float(tv) / float(tqty)
                except Exception:
                    pass

                partner = dataset['ptTitle']
                partnerCode = dataset['pt3ISO']
                
                # Implement logic for saving data
                years_list.append(year)
                reporter_list.append(country['text'])
                reporterCode_list.append(country['id'])
                partner_list.append(partner)
                partnerCode_list.append(partnerCode)
                cc_list.append(cc)
                tradeValue_list.append(tv)
                tradeQty_list.append(tqty)
                TVperUnit_list.append(tv_per_unit)
                
                print(f"{year} \t REPORTER :: [{country['id']} | {country['text']}] [{classification_codes[cc]} | {cc}] --- PARTNER :: [{partnerCode} | {partner}] \t\t --> Tr.V.: {tv}")
                time.sleep(5)
            # # Add aggregated data in case dataset contains at least one element --> NOT NECESSARY BECAUSE THERE IS A ROW UNDER THE NAME "WORLD" WITH THAT NAME.
            # if len(datasets) >= 1:
            #     years_list.append(year)
                
            #     reporter_list.append(country['text'])
            #     reporterCode_list.append(country['id'])
            #     partner_list.append('ALL')
            #     partnerCode_list.append('all')
                
            #     cc_list.append(cc)
            #     tradeValue_list.append(total_tv)
            #     tradeQty_list.append(total_tqty)
            #     try:
            #         TVperUnit_list.append(float(total_tv) / float(total_tqty))
            #     except Exception:
            #         TVperUnit_list.append(-2) # @CODE ERROR [-2] for indicating that something happened in the aggregation
                
            #     print(f"{year} \t REPORTER :: [{country['id']} | {country['text']}] [{classification_codes[cc]} | {cc}] --- PARTNER :: [all | ALL] \t\t --> Tr.V.: {tv}")
            

    stage_df = pd.DataFrame({
        "Year": years_list,
        "Reporter": reporter_list,
        "ReporterCode": reporterCode_list,
        "Partner": partner_list,
        "PartnerCode": partnerCode_list,
        # "CountryCode": countryCodes_list,
        "CC": cc_list,
        "TradeValue": tradeValue_list,
        "TradeQuantity": tradeQty_list,
        "TVperUnit": TVperUnit_list
    })
    stage_df.to_csv(f"../data/imports_values/stage_{year}.csv", index=False)


[get_trade_value_imports][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
[get_trade_value_imports][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
[get_trade_value_imports][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
[get_trade_value_imports][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
[get_trade_value_imports][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
[get_trade_value_imports][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
[get_trade_value_imports][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
[get_trade_value_imports][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
[get_trade_value_imports][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
[get_trade_value_imports][ERROR] Something happened: Expecting value: line 1 column 1 (char 0)
[get_trade_value_imports][ERROR] Something happene

In [11]:
list_reporter_countries

[{'id': '4', 'text': 'Afghanistan'},
 {'id': '8', 'text': 'Albania'},
 {'id': '12', 'text': 'Algeria'},
 {'id': '20', 'text': 'Andorra'},
 {'id': '24', 'text': 'Angola'},
 {'id': '660', 'text': 'Anguilla'},
 {'id': '28', 'text': 'Antigua and Barbuda'},
 {'id': '32', 'text': 'Argentina'},
 {'id': '51', 'text': 'Armenia'},
 {'id': '533', 'text': 'Aruba'},
 {'id': '36', 'text': 'Australia'},
 {'id': '40', 'text': 'Austria'},
 {'id': '31', 'text': 'Azerbaijan'},
 {'id': '44', 'text': 'Bahamas'},
 {'id': '48', 'text': 'Bahrain'},
 {'id': '50', 'text': 'Bangladesh'},
 {'id': '52', 'text': 'Barbados'},
 {'id': '112', 'text': 'Belarus'},
 {'id': '56', 'text': 'Belgium'},
 {'id': '58', 'text': 'Belgium-Luxembourg'},
 {'id': '84', 'text': 'Belize'},
 {'id': '204', 'text': 'Benin'},
 {'id': '60', 'text': 'Bermuda'},
 {'id': '64', 'text': 'Bhutan'},
 {'id': '68', 'text': 'Bolivia (Plurinational State of)'},
 {'id': '535', 'text': 'Bonaire'},
 {'id': '70', 'text': 'Bosnia Herzegovina'},
 {'id': '72